In [10]:
import pandas as pd
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


data = pd.read_csv('/Users/Boaz/Documents/DataMining/prepro_data_RF.csv')
data = data.drop('time',axis = 1)
data = data.drop('season',axis =1 )
data = data.drop('id', axis = 1)
data['mood'] = round(data['mood'])
data['circumplex.arousal'] = round(data['circumplex.arousal'])
data['circumplex.valence'] = round(data['circumplex.valence'])
data['target_mood_plus1'] = round(data['target_mood_plus1'])

data

,mood,circumplex.arousal,circumplex.valence,activity,screen,sms,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,target_mood_plus1,summer,spring,winter
0,6,-0.0,1.0,0.000000,0.000000,2.0,0.000,0.000,0.000,0.000,...,0.000,0.0000,0.000,0.0,0.000,0.000,6.0,0,0,1
1,6,0.0,0.0,3.083152,9100.601001,0.0,1255.653,5903.482,1007.456,49.544,...,139.195,3945.8130,915.445,0.0,598.754,0.000,6.0,0,1,0
2,6,1.0,0.0,3.790084,6142.161000,1.0,480.124,4962.918,93.324,21.076,...,98.143,439.6320,37.305,0.0,117.621,0.000,7.0,0,1,0
3,6,0.0,1.0,2.141117,6033.096002,0.0,615.324,4495.049,94.346,43.403,...,72.823,900.8390,0.000,0.0,30.086,30.386,6.0,0,1,0
4,6,1.0,0.0,1.895065,10422.503001,0.0,816.953,6809.575,417.651,34.106,...,66.558,2644.4390,54.701,0.0,178.732,0.000,7.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,6,-1.0,0.0,3.713814,6659.054999,1.0,1282.030,1567.344,194.216,0.000,...,42.362,2868.6710,0.000,0.0,96.982,0.000,5.0,1,0,0
1207,5,-0.0,0.0,1.497619,5975.597999,3.0,846.084,1235.537,688.769,0.000,...,151.582,5441.2640,0.000,0.0,0.000,0.000,6.0,1,0,0
1208,6,-1.0,0.0,0.304902,3377.526001,2.0,365.069,2078.403,124.694,0.000,...,142.686,2010.3640,0.000,0.0,56.173,0.000,8.0,1,0,0
1209,8,0.0,1.0,2.479228,6241.620998,1.0,1751.404,1936.415,614.706,0.000,...,336.913,4487.4515,0.000,0.0,30.666,0.000,7.0,1,0,0


In [11]:
from collections import Counter
count = Counter(data['target_mood_plus1'])
count



Counter({6.0: 253, 7.0: 613, 8.0: 303, 4.0: 8, 5.0: 23, 9.0: 9, 3.0: 2})

In [12]:
data = data[data.target_mood_plus1 != 3]
count = Counter(data['target_mood_plus1'])
count

Counter({6.0: 253, 7.0: 613, 8.0: 303, 4.0: 8, 5.0: 23, 9.0: 9})

In [13]:
from sklearn.model_selection import train_test_split
X_train_total = []
X_test_total = []
Y_train_total = []
Y_test_total = []
labels = np.array(data['target_mood_plus1'])
features= data.drop('target_mood_plus1', axis = 1)
features = np.array(features)
Xtrain, Xtest, ytrain, ytest = train_test_split(features, labels, random_state = 42,stratify=labels)
X_train_total.append(Xtrain)
X_test_total.append(Xtest)
Y_train_total.append(ytrain)
Y_test_total.append(ytest)


In [14]:
# GET RID OF NESTED LIST
X_train_total = [item for l in X_train_total for item in l]
X_test_total = [item for l in X_test_total for item in l]
Y_train_total = [item for l in Y_train_total for item in l]
Y_test_total = [item for l in Y_test_total for item in l]

In [15]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [17]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [18]:
rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [8, 10, 12],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2, 3],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [2000,1800, 2200]
}
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [20]:
grid_search.fit(X_train_total, Y_train_total)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  2.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [21]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 2000}

In [25]:
best_grid = grid_search.best_estimator_
best_grid
# best_grid.fit(X_train_total, Y_train_total)
# predictions = best_grid.predict(X_test_total)
# predictions = [np.argmax(p) for p in predictions]
#predictions

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [23]:
predictions = predictions.round()
print(accuracy_score(Y_test_total, predictions))
print(classification_report(Y_test_total, predictions))
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_squared_error(Y_test_total, predictions))

0.5313531353135313
              precision    recall  f1-score   support

         4.0       0.00      0.00      0.00         2
         5.0       0.00      0.00      0.00         6
         6.0       0.44      0.17      0.25        63
         7.0       0.53      0.94      0.68       154
         8.0       0.75      0.08      0.14        76
         9.0       0.00      0.00      0.00         2

    accuracy                           0.53       303
   macro avg       0.29      0.20      0.18       303
weighted avg       0.55      0.53      0.43       303

0.5808580858085809


C:\Users\Boaz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# Get numerical feature importances
features = data.drop('target_mood_plus1', axis = 1)
feature_list = list(features.columns)

importances = list(best_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: mood                 Importance: 0.15
Variable: activity             Importance: 0.09
Variable: screen               Importance: 0.08
Variable: appCat.builtin       Importance: 0.08
Variable: appCat.communication Importance: 0.08
Variable: appCat.entertainment Importance: 0.08
Variable: appCat.other         Importance: 0.07
Variable: appCat.social        Importance: 0.07
Variable: circumplex.valence   Importance: 0.04
Variable: appCat.travel        Importance: 0.04
Variable: circumplex.arousal   Importance: 0.03
Variable: sms                  Importance: 0.03
Variable: appCat.office        Importance: 0.03
Variable: appCat.utilities     Importance: 0.03
Variable: appCat.finance       Importance: 0.02
Variable: appCat.game          Importance: 0.02
Variable: appCat.unknown       Importance: 0.02
Variable: appCat.weather       Importance: 0.02
Variable: summer               Importance: 0.01
Variable: spring               Importance: 0.01
Variable: winter               Importanc

In [26]:
best_grid = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

best_grid.fit(X_train_total, Y_train_total)
predictions = best_grid.predict(X_test_total)
# predictions = [np.argmax(p) for p in predictions]
predictions = predictions.round()
print(accuracy_score(Y_test_total, predictions))
print(classification_report(Y_test_total, predictions))
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_absolute_error(Y_test_total, predictions))


0.48184818481848185
              precision    recall  f1-score   support

         4.0       0.00      0.00      0.00         2
         5.0       0.00      0.00      0.00         6
         6.0       0.13      0.03      0.05        63
         7.0       0.50      0.94      0.65       154
         8.0       0.00      0.00      0.00        76
         9.0       0.00      0.00      0.00         2

    accuracy                           0.48       303
   macro avg       0.11      0.16      0.12       303
weighted avg       0.28      0.48      0.34       303

0.5511551155115512


C:\Users\Boaz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
